In [ ]:
# MA20 回測專案

## 專案目的
用 Python 對台股（yfinance）進行 MA20 策略回測，並輸出各股票績效報表。

## 策略規則（MA20）
- 當日 Close > MA20 → 下一交易日持有（position_shift）
- 當日 Close <= MA20 → 下一交易日空手
- 交易成本：每次換倉扣 cost_rate

## 產出結果
- 每檔股票一份 report.csv
- 合併總表 ALL_report.csv


In [ ]:
## Debug / 踩雷紀錄

- `KeyError: equity_bh`：原因是我在算 MDD 前沒有先建立 equity 曲線。
- `trade` 計算 bug：用 diff() 會誤判，改成用「持倉是否切換」來算交易次數。
- `FileNotFoundError`：原因是路徑或檔名規則沒統一，後來統一輸出資料夾與檔名格式。

In [1]:
import os
import numpy as np
import pandas as pd
import yfinance as yf


In [ ]:
# 回測的股票
stocks = ["2330.TW", "2317.TW", "2603.TW"]

# 回測時間
start_date = "2020-01-01"
end_date   = "2025-12-31"

# 策略參數
ma_window = 20
cost_rate = 0.001  

# 資料夾設定
project_dir = r"C:\Users\USER\Desktop\複習專案"

raw_dir      = os.path.join(project_dir, "data", "raw")
clean_dir    = os.path.join(project_dir, "data", "clean")
features_dir = os.path.join(project_dir, "outputs", "features")
report_dir   = os.path.join(project_dir, "outputs", "reports")

for d in [raw_dir, clean_dir, features_dir, report_dir]:
    os.makedirs(d, exist_ok=True)


In [5]:
#下載資料函數
def download_stock(stocks, start=start_date, end=end_date):
    df = yf.download(stocks, start=start, end=end, progress=False)# 不要顯示下載進度條 progress

    # 有時候 yfinance 會產生 MultiIndex，直接壓平
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    df = df.reset_index()
    return df


In [ ]:
#清洗資料函數
def clean_stock(df):
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values("Date")
    df = df.set_index("Date")
    return df


In [8]:
#報酬計算
def add_returns(df):
    df["ret"] = df["Close"].pct_change()
    return df


In [ ]:
#MA20策略
def add_ma20_strategy(df, ma=20, cost=0.001):
    df = df.copy()

    # MA
    df["ma20"] = df["Close"].rolling(ma).mean()

    # 今日訊號（收盤後才知道）
    df["position"] = np.where(df["Close"] > df["ma20"], 1, 0)

    # 實際持倉（避免偷看未來）
    df["position_shift"] = df["position"].shift(1).fillna(0)

    # 策略日報酬
    df["strategy_ret"] = df["position_shift"] * df["ret"]

    # 用「持倉是否切換」算交易次數
    df["trade"] = (df["position_shift"] != df["position_shift"].shift(1)).astype(int)
    df.loc[df.index[0], "trade"] = 0

    # 扣成本：有交易就扣
    df["strategy_ret_cost"] = df["strategy_ret"] - df["trade"] * cost

    return df


In [10]:
#績效函數
def perf_from_ret(ret, name, freq=252):
    ret = ret.dropna()
    equity = (1 + ret).cumprod()

    total_return = equity.iloc[-1] - 1
    n = len(ret)

    ann_return = (1 + total_return) ** (freq / n) - 1 if n > 0 else np.nan
    ann_vol = ret.std() * np.sqrt(freq)

    sharpe = (ret.mean() * freq) / (ret.std() * np.sqrt(freq)) if ret.std() != 0 else np.nan

    rolling_max = equity.cummax()
    mdd = (equity / rolling_max - 1).min()

    return pd.Series({
        "TotalReturn": total_return,
        "AnnReturn": ann_return,
        "AnnVol": ann_vol,
        "Sharpe": sharpe,
        "MDD": mdd
    }, name=name)


In [11]:
def run_one_stock(ticker):
    raw = download_stock(ticker)
    df = clean_stock(raw)
    df = add_returns(df)
    df = add_ma20_strategy(df, ma=ma_window, cost=cost_rate)

    # 三種報酬
    report = pd.concat([
        perf_from_ret(df["ret"], "Buy&Hold"),
        perf_from_ret(df["strategy_ret"], "MA20"),
        perf_from_ret(df["strategy_ret_cost"], "MA20_with_cost")
    ], axis=1).T

    report["ticker"] = ticker
    report = report.reset_index(names="Strategy").set_index(["ticker", "Strategy"])

    return df, report


In [13]:
all_reports = []

for s in stocks:
    print(f"處理中：{s}")

    df, report = run_one_stock(s)

    # 存 features（你要也可以不存）
    features_fp = os.path.join(features_dir, f"{s}_features.csv")
    df.reset_index().to_csv(features_fp, index=False, encoding="utf-8-sig")

    # 存單檔 report
    report_fp = os.path.join(report_dir, f"{s}_report.csv")
    report.to_csv(report_fp, encoding="utf-8-sig")

    all_reports.append(report)

ALL_report = pd.concat(all_reports)
all_fp = os.path.join(report_dir, "ALL_report.csv")
ALL_report.to_csv(all_fp, encoding="utf-8-sig")

ALL_report


處理中：2330.TW
處理中：2317.TW


C:\Users\USER\AppData\Local\Temp\ipykernel_9860\1778457011.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stocks, start=start, end=end, progress=False)# 不要顯示下載進度條 progress
C:\Users\USER\AppData\Local\Temp\ipykernel_9860\1778457011.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stocks, start=start, end=end, progress=False)# 不要顯示下載進度條 progress


處理中：2603.TW


C:\Users\USER\AppData\Local\Temp\ipykernel_9860\1778457011.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(stocks, start=start, end=end, progress=False)# 不要顯示下載進度條 progress


TotalReturn  AnnReturn    AnnVol    Sharpe       MDD
ticker  Strategy                                                            
2330.TW Buy&Hold           4.060749   0.324241  0.297194  1.093415 -0.447990
        MA20               3.171279   0.280643  0.211499  1.274796 -0.236071
        MA20_with_cost     2.565472   0.246305  0.211548  1.145963 -0.274094
2317.TW Buy&Hold           2.207508   0.223675  0.305648  0.813269 -0.503311
        MA20               1.090700   0.136248  0.208377  0.716062 -0.482903
        MA20_with_cost     0.726709   0.099222  0.208675  0.556514 -0.543305
2603.TW Buy&Hold          15.253177   0.620800  0.522901  1.185897 -0.705367
        MA20              14.823584   0.613298  0.396394  1.404848 -0.447450
        MA20_with_cost    12.654626   0.572627  0.396692  1.339614 -0.453617